In [1]:
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
import pickle

In [2]:
def HOG_(image):
    hog_feature = hog(image, orientations=30, pixels_per_cell=(6, 6), cells_per_block=(1, 1), block_norm='L2-Hys',transform_sqrt=True)
    return hog_feature

def LBP_(image):
    lbp_feature=local_binary_pattern(image,8,1)
    hist_lbp, _ = np.histogram(lbp_feature, bins=int(lbp_feature.max() + 1), range=(0,int(lbp_feature.max() + 1)),density=True)
    return hist_lbp

def GLCM_(image):
    glcm = np.zeros((256, 256), dtype=np.uint8)
    height, width = image.shape
    for i in range(height - 1):
        for j in range(width - 1):
            p = image[i, j]
            q = image[i, j + 1]
            glcm[p, q] += 1
    glcm = glcm.astype(np.float64)
    glcm /= np.sum(glcm)  
    contrast = np.sum(np.square(glcm - np.mean(glcm)))
    dissimilarity = np.sum(np.abs(glcm - np.mean(glcm)))
    homogeneity = np.sum(glcm / (1.0 + np.abs(i - j)))
    energy = np.sum(np.square(glcm))
    correlation = np.sum(np.divide((i - np.mean(glcm)) * (j - np.mean(glcm)), np.sqrt(np.sum(np.square(i - np.mean(glcm)))) * np.sqrt(np.sum(np.square(j - np.mean(glcm))))))
    glcm_feature = np.array([contrast, dissimilarity, homogeneity, energy, correlation])
    return glcm_feature

In [ ]:
cap=cv2.VideoCapture("C:\\Users\\ASUS\\Desktop\\U2.mp4")
facedetector=cv2.CascadeClassifier('C:\\Users\\ASUS\\Desktop\\haarcascade_frontalface_default.xml')
vwriter=cv2.VideoWriter('out.wmv',cv2.VideoWriter_fourcc(*'WMV1'),20,(640,480))
with open('C:\\Users\\ASUS\\Desktop\\:).pkl','rb') as f:
    clf = pickle.load(f)
while(True):
    ret,frame=cap.read()
    if ret:
        gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces=facedetector.detectMultiScale(gray,1.1,4)
        for (x,y,w,h) in faces:
            face = cv2.resize(frame, (100, 100), interpolation=cv2.INTER_AREA)
            hog_face = []
            lbp_face = []
            glcm_face = []
            hog_face.append(HOG_(face))
            lbp_face.append(LBP_(face))
            glcm_face.append(GLCM_(face))
            test_feature = np.concatenate((hog_face, lbp_face, glcm_face), axis=1)
            ans=clf.predict(test_feature)
            if(ans[0]):
               cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            else:
                cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        vwriter.write(frame)
        cv2.imshow('frame',frame)
        cv2.waitKey(10)
    else:
        vwriter.release()
cap.release()
